# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sklearn
import tensorflow as tf
KERAS_BACKEND=tf
from tensorflow import keras
# from tensorflow.keras.utils import plot_model
import gensim
# from keras_self_attention import SeqSelfAttention
import keras
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Input, Embedding, Bidirectional, LSTM
from gensim.models.keyedvectors import KeyedVectors



Using TensorFlow backend.


In [2]:
tf.__version__

'2.3.0'

# Loading Yoga Class & Pose DataFrames
These DataFrames were scraped and created during project 3. 

In [3]:
f = open("all_poses", "rb")
all_poses = pickle.load(f)
f.close()

f = open("flask_app_df", "rb")
poses_info = pickle.load(f)
f.close()

f = open("all_yoga_classes_df", "rb")
yoga_classes = pickle.load(f)
f.close()

f = open("poses_df", "rb")
class_poses = pickle.load(f)
f.close()

f = open("vinyasa_df", "rb")
more_vinyasa = pickle.load(f)
f.close()

f = open("hatha_df", "rb")
more_hatha = pickle.load(f)
f.close()

In [4]:
yoga_classes

,Title,Poses,Class Type
0,←Slow Sunday Flow y Monday early,"[Easy Pose Hands To Heart, Easy Pose Hands Int...",Vinyasa
1,←Anahata,"[Mantra Section, Thunderbolt Pose, Easy Pose B...",Vinyasa
2,←CORE,"[Classic Sun Salutation Variation F, Chair Pos...",Vinyasa
3,←,"[Easy Pose, Easy Pose Warm Up Flow, Sun Saluta...",Vinyasa
4,←Vinyasa - Bench press & push up #,"[Corpse Pose, Corpse Pose Roll Under Spine, Wr...",Vinyasa
...,...,...,...
1582,←Shoulder-Heart Opener,"[Easy Pose, Easy Pose Neck Side Stretch, Easy ...",Yin
1583,←Yin Sequence,"[Easy Pose, Cat Cow Pose, Child Pose, Seated S...",Yin
1584,←Practicing Presence,"[Easy Pose Hands To Heart, Easy Pose Twist Rai...",Yin
1585,←Yin- Cooling,"[Sitali, Bound Angle Pose Blocks, Sun Breaths,...",Yin


### Additional Classes
I scraped 20,000 additional classes since project 3, to supplement the data. In particular, they are vinyasa and hatha classes.

In [5]:
more_vinyasa

,Title,Poses,Class Type
0,←Foundations -,"[Easy Pose, Easy Pose Elbows On Floor, Easy Po...",Vinyasa
1,←Massage therapist,"[Mountain Pose, Palm Tree Pose, Palm Tree Pose...",Vinyasa
2,←VIN I/II mins,"[Easy Pose, Seated Neck Rolls, Easy Pose Neck ...",Vinyasa
3,←Core Feather,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Vinyasa
4,←Vinyasa for Shoulders,"[Reclined Arms Movement Flow, Supine Snow Ange...",Vinyasa
...,...,...,...
3811,←Back and Legs Sequence,"[Child Pose, Table Top Child Pose Flow, Cat Co...",Vinyasa
3812,←Vinyasa L-Stand,"[Corpse Pose, Pranayama, Supine Windshield Wip...",Vinyasa
3813,←inver,"[Reclined Crow Pose, Reclined Eagle Crunches, ...",Vinyasa
3814,←Basics Class for Hip Opening,"[Easy Pose Block, Seated Cat Cow Pose, Mountai...",Vinyasa


In [6]:
more_hatha

,Title,Poses,Class Type
0,←Low Back & Hamstrings //,"[Seated Straddle Pose Knees Bent Bolster, Easy...",Hatha
1,←Basic Yoga Sequenz (YTT),"[Prop Strap, Prop Block, Three Part Breath Cor...",Hatha
2,←evening,"[Easy Pose, Easy Pose Variation Side Bend, Eas...",Hatha
3,←Standing class warm-up,"[Mountain Pose, Mountain Pose Namaste, Palm Tr...",Hatha
4,←Sequence #,"[Wind Release Pose, Half Wind Release Pose, Ha...",Hatha
...,...,...,...
4623,←Seri,"[Standing Cat Cow Pose, Downward Facing Dog Po...",Hatha
4624,←Flexibility I,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Hatha
4625,←Postnatal-,"[Kneeling Pose, Cat Cow Pose Forearms, Hoverin...",Hatha
4626,←Herbst - Flow,"[Mountain Pose, Chair Pose Deep Bend, Standing...",Hatha


In [7]:
df = pd.concat([yoga_classes, more_vinyasa, more_hatha])

In [8]:
df.shape

(63337, 3)

### Excluding Restorative Class Types
I am excluding the classes that are of type "gentle", "restorative" and "yin", as these are very slow restorative/laying down type classes that I'm not aiming to create (yet) with this app. I want more dynamic, flow and exercise based classes to be generated, plus I have much more data availble in support of creating those types. 

In [9]:
df = df.loc[df["Class Type"].isin(["Vinyasa", "Hatha", "Power", "Iyengar", "Ashtanga"])]

In [10]:
df.reset_index(inplace=True)

In [11]:
df

,index,Title,Poses,Class Type
0,0,←Slow Sunday Flow y Monday early,"[Easy Pose Hands To Heart, Easy Pose Hands Int...",Vinyasa
1,1,←Anahata,"[Mantra Section, Thunderbolt Pose, Easy Pose B...",Vinyasa
2,2,←CORE,"[Classic Sun Salutation Variation F, Chair Pos...",Vinyasa
3,3,←,"[Easy Pose, Easy Pose Warm Up Flow, Sun Saluta...",Vinyasa
4,4,←Vinyasa - Bench press & push up #,"[Corpse Pose, Corpse Pose Roll Under Spine, Wr...",Vinyasa
...,...,...,...,...
48113,4623,←Seri,"[Standing Cat Cow Pose, Downward Facing Dog Po...",Hatha
48114,4624,←Flexibility I,"[Mountain Pose, Mountain Pose Raised Hands Hea...",Hatha
48115,4625,←Postnatal-,"[Kneeling Pose, Cat Cow Pose Forearms, Hoverin...",Hatha
48116,4626,←Herbst - Flow,"[Mountain Pose, Chair Pose Deep Bend, Standing...",Hatha


Looking at the average length of classes so I know what length of class I should typically be generating.

In [12]:
# average length of classes
lengths = []
for i in range(len(df["Poses"])):
    length = len(df["Poses"][i])
    lengths.append(length)
print(np.average(length))

33.0


# Tokenization
First I am making the lists of poses that comprise each class into the type of input Word2Vec or a neural net is looking for. Effectively I am tokenizing manually, instead of using a "Tokenizer", becuase I want to break up by each pose, not by each word etc. 

In [13]:
documents = [class_list for class_list in df["Poses"]]
documents[0:2]

[['Easy Pose Hands To Heart',
  'Easy Pose Hands Interlocked Behind Head',
  'Seated Torso Circles',
  'Easy Pose Variation Side',
  'Half Easy Pose Half Forward Bend',
  'Seated Windshield Wiper Pose Chest Opener Variation',
  'Cat Cow Pose',
  'Child Pose Hip Circles Flow',
  'Thread The Needle Pose Flow',
  'Downward Facing Dog Pose',
  'Downward Facing Dog Pose Plank Pose Flow',
  'Right Leg Forward',
  'Three Legged Downward Facing Dog Pose Prep',
  'Fallen Triangle Pose Variation',
  'Repeat Other Side',
  'Right Leg Forward',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Hands To Knee',
  'Low Lunge Pose Open Twist',
  'Revolved Low Lunge Pose Knee On The Floor',
  'Runners Lunge Pose',
  'Revolved Lunge Pose',
  'Extended Side Angle Pose Variation Hand In Front',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Hands To Knee',
  'Left Leg Forward',
  'Repeat Other Side',
  'Top Of Mat Section',
  'Mountain Pose',
  'Crescent High Lunge Pose Arms On Hips Flow',


# Word2Vec
I am creating word embeddings with Word2Vec via gensim. I'm training the model on the yoga classes to create vectors of the similarities between different yoga poses, based on how close together (sequentially) they're featured within the yoga classes. I.e. output = pose embeddings. 

{Choosing to keep poses which show up in at least 50 classes (out of 48000).}

In [14]:
model = gensim.models.Word2Vec(documents, size=100, window=5, min_count=50, sg=1)

In [15]:
len(list(model.wv.vocab.items()))

2434

In [16]:
print(model["Cat Cow Pose"])

[ 0.01223068 -0.01918484  0.22792435 -0.17332345 -0.17573859 -0.26205277
  0.4005988   0.24614489 -0.32392177  0.04885524  0.5436343  -0.16210674
 -0.09152774  0.29717615  0.02620299  0.16229303  0.00875798 -0.05532655
  0.12045298  0.01618364  0.04564977  0.21082433 -0.2854309   0.16240077
 -0.18996267 -0.11400176 -0.2791333  -0.14880304 -0.00755632 -0.06774661
 -0.05787782  0.2336689   0.25875437  0.06112934 -0.12278202 -0.17318788
  0.04721717 -0.20559984  0.08359534  0.03315291 -0.16931503 -0.14190222
 -0.38389263 -0.20836148  0.0245751  -0.00996474 -0.35866165 -0.48580843
  0.32020372 -0.07933161  0.00739082  0.00572685  0.05135494  0.49267042
  0.05806376 -0.33078358 -0.13434118  0.5331267  -0.14256062  0.1276749
  0.03440032 -0.41323403  0.15787965  0.09662225 -0.36112002  0.3391559
 -0.21151225 -0.1554054   0.17167422  0.4080536   0.16002001 -0.4708573
  0.00607343  0.05668337  0.05060999 -0.09178577 -0.42604646 -0.16699961
 -0.02113839 -0.41327912 -0.14550409 -0.31743342 -0.31

In [17]:
model.most_similar("Downward Facing Dog Pose", topn=20)

[('Three Legged Downward Facing Dog Pose Tiger Curl Pose Flow',
  0.7108982801437378),
 ('Plank Pose', 0.7103734016418457),
 ('Three Legged Downward Facing Dog Pose', 0.6983637809753418),
 ('Downward Facing Dog Pose Variation Knees Bent', 0.682325005531311),
 ('Low Lunge Pose', 0.681185245513916),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.672895610332489),
 ('Runners Lunge Pose', 0.6680295467376709),
 ('Three Legged Downward Facing Dog Pose Variation Stacked',
  0.6573711633682251),
 ('Standing Forward Fold Pose', 0.6501185894012451),
 ('Table Top Pose', 0.6500736474990845),
 ('Wide Child Pose', 0.6328990459442139),
 ('Four Limbed Staff Pose', 0.6284885406494141),
 ('Child Pose', 0.6263731718063354),
 ('Revolved Lunge Pose', 0.6249303817749023),
 ('Low Lunge Pose Hands On Hips', 0.6151965856552124),
 ('Crescent Low Lunge Pose Variation Knee On Floor', 0.6030289530754089),
 ('Low Lunge Hands To Knee', 0.59831303358078),
 ('Upward Forward Fold Hands On Shins', 0.5962111949920654),


In [18]:
model.most_similar("Corpse Pose")

[('Corpse Pose (IRT)', 0.7784069776535034),
 ('Corpse Pose Variation Bolster', 0.7354565262794495),
 ('Corpse Pose Roll Under Spine', 0.7238022685050964),
 ('Savasana Section', 0.7216494083404541),
 ('Wind Release Pose', 0.7187507152557373),
 ('Corpse Pose Blanket', 0.7031890153884888),
 ('Deep Relaxation Technique', 0.6820505857467651),
 ('Supine Neck Massage Pose', 0.6702317595481873),
 ('Pentacle Pose', 0.6658196449279785),
 ('Wind Release Pose Nose To Knee', 0.6622316241264343)]

Problem is a lot of the above aren't even really variations... they should probably be reduced to their base pose before applying the word embedding model. 

In [19]:
model.most_similar("Plank Pose")

[('Four Limbed Staff Pose', 0.7338338494300842),
 ('Plank Pose Knees Bent', 0.7251478433609009),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.7178030610084534),
 ('Downward Facing Dog Pose', 0.7103734016418457),
 ('Plank Pose Variation Knee To Ground (Variation)', 0.6980160474777222),
 ('Runners Lunge Pose', 0.6661559343338013),
 ('Plank Pose Four Limbed Staff Pose Flow', 0.6661163568496704),
 ('Plank Pose Shoulder Taps', 0.6394550204277039),
 ('Four Limbed Staff Pose Knees To Floor Variation', 0.6393851041793823),
 ('Upward Facing Dog Pose', 0.6392515897750854)]

In [20]:
model.most_similar("Wheel Pose", topn=20)

[('Wheel Pose Prep', 0.7680707573890686),
 ('One-Legged Wheel Pose', 0.7465466260910034),
 ('Two-legged Inverted Staff Pose', 0.662020206451416),
 ('One Legged Inverted Staff Pose', 0.6557153463363647),
 ('Wheel Pose Block Between Knees', 0.6412934064865112),
 ('Bridge Pose On Head', 0.6260730624198914),
 ('Bridge Pose Variation Arms Support Legs Straight', 0.6209083795547485),
 ('Fish Pose Variation Lotus Elbows', 0.6107114553451538),
 ('One-Legged Bridge Pose', 0.6070903539657593),
 ('Plough Pose', 0.6041018962860107),
 ('Shoulderstand Seal Pose', 0.5983352065086365),
 ('Shoulderstand Pose', 0.5977904200553894),
 ('Bridge Pose Variation Bound Hands', 0.5965489149093628),
 ('Wheel Pose Wall Block Under Hands I', 0.5880383253097534),
 ('Ear Pressure Pose', 0.5825566053390503),
 ('Upward Lotus Pose', 0.5768963098526001),
 ('Half Shoulderstand Pose', 0.5762155652046204),
 ('Bridge Pose Variation Hands Support', 0.5728883743286133),
 ('Unsupported Shoulder Stand Pose', 0.5700054168701172)

In [21]:
model.most_similar("Sun Salutation A")

[('Sun Salutation B', 0.8557642102241516),
 ('Classic Sun Salutation Variation E', 0.6979700326919556),
 ('Sun Salutation A Variation', 0.6805431842803955),
 ('Classic Sun Salutation Variation Ii', 0.6737818121910095),
 ('Classic Sun Salutation Variation', 0.6294876337051392),
 ('Sun Salutation', 0.6043938398361206),
 ('Sun Salutation A First Half', 0.5925980806350708),
 ('Mountain Pose', 0.577546238899231),
 ('Sun Breath Yoga Sequence', 0.5767609477043152),
 ('Dangling Pose', 0.5722558498382568)]

In [22]:
print(model.similarity("Corpse Pose", "Hand to Big Toe Pose"))
print(model.similarity("Corpse Pose", "Pigeon Pose"))
print(model.similarity("Corpse Pose", "Child Pose"))
print(model.similarity("Corpse Pose", "Wind Release Pose"))

0.07074697
0.2299179
0.36891097
0.7187508


Honestly that looks great? Very low similarity between corpse pose and an intense standing/balance pose, slightly higher for an intense ground stretch, moderate similarity for another gentle pose but in a very different body orientation, and highestest for a similar supine relaxation pose. 

However, there are currently a couple thousand total pose variations, many of which aren't even really distinct poses. I set the minimum at a range of values, higher meant to exclude less common poses, but it is losing key information -- even set at just 100, I lost "pigeon pose" which is very common. This is probably because it is listed with slightly varying names across classes. Hence, instead of just deleting poses if they don't show up often enough, I want to reduce all the variations/duplicate name variations to their base poses, which will then show up far more often and be more useful.

Besides, for the practicality of my app and creating animations to accompany each pose (which I, at least for now, anticpate having to hand pose each one), I can't accomodate thousands of poses, only a few hundred. 

For that reason, I'm going to reduce the poses in each class to their base poses before doing word embeddings. 

## Reducing to Base Poses
As outlined above, I am reducing all poses to their base poses and key variations. 

I am doing this primarily just for the sake of the MVP animation -- I have no doubt the neural network would handle even thousands of poses well, select the most importnat, and create even more diverse classes, but I will have to hand animate every single pose for now since I don't know much about Unity. I can't practically do that for thousands of poses, so I am limiting the total possible number of poses to less than 200. 

I first want to isolate just information about poses and base poses, for simplification of all the poses in classes from thousands of variations down to a usable size for creating classes. From the poses_info DataFrame (scraped and created during project 3):

In [23]:
poses_info.head(3)

,Base Pose,Level,Pose Name,Back-Bend,Balance,Forward-Bend,Inversion,Meditative,Pranayama,Restorative,...,Standing,Supine,Toes,Crown Chakra (Sahasrara Chakra),Heart Chakra (Anahata Chakra),Root Chakra (Muladhara Chakra),Sacral Chakra (Swadisthana Chakra),Solar Plexus (Manipura Chakra),Third Eye Chakra (Ajna Chakra),Throat Chakra (Vishuddha Chakra)
0,Boat Pose,Advanced,Boat Pose Aerial,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Boat Pose,Advanced,Boat Pose Twist Variation Hands To Feet,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Boat Pose,Advanced,Elevated Boat Pose,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
base_poses = poses_info[["Pose Name", "Base Pose"]]
base_poses

,Pose Name,Base Pose
0,Boat Pose Aerial,Boat Pose
1,Boat Pose Twist Variation Hands To Feet,Boat Pose
2,Elevated Boat Pose,Boat Pose
3,Half Bound Lotus Boat Pose,Boat Pose
4,Bicycle Pose,Boat Pose
...,...,...
3211,Half Wind Release Pose Nose To Knee Foot Raised,Wind Release Pose
3212,Hip Opening Yoga Sequence Hippie Flow,Wind Release Pose
3213,Reclined Cow Face Pose,Wind Release Pose
3214,Seated Broken Wings Pose,Wind Release Pose


In [25]:
base_poses["Base Pose"].unique()

array(['Boat Pose', 'Bound Angle Pose', 'Bow Pose', 'Bridge Pose',
       'Camel Pose', 'Cat Cow Pose', 'Cat Tail Pulling Pose',
       'Center Splits Pose', 'Chair Pose', 'Child Pose', 'Cobra Pose',
       'Corpse Pose', 'Cow Face Pose', 'Crane Pose', 'Crow Pose',
       'Cycling Pose', 'Dancer Pose', 'Dolphin Pose',
       'Downward Facing Dog Pose', 'Eagle Pose', 'Easy Pose',
       'Easy Pose Chair', 'Exercise', 'Extended Side Angle Pose',
       'Eye Exercise', 'Eye Of The Needle Pose', 'Feathered Peacock Pose',
       'Firelog Pose', 'Fish Pose', 'Flamingo Pose',
       'Foot Behind The Head Pose', 'Four Limbed Staff Pose',
       'Garland Pose', 'Gate Pose', 'Goddess Pose',
       'Half Lord Of The Fishes Pose', 'Half Moon Pose', 'Handstand Pose',
       'Happy Baby Pose', 'Head To Knee Pose', 'Headstand Pose',
       'Hero Pose', 'High Lunge Pose', 'Intense Leg Stretch Pose',
       'Intense Side Stretch Pose', 'Legs Up The Wall Pose', 'Lion Pose',
       'Lizard Pose', 'Locust

Currently there are only 102 total base poses for all 3200 poses I collected, which seems like a bit too much of a simplification. I imagine there are certain key variations that are distinct enough that I'd want to keep them -- hence have more than 102 poses to build my classes from. So, I'm going to manually go through all 3200 poses and make modification to the associated base pose, sometimes choosing to keep the distinct variations instead, using my domain knowledge as a certified yoga teacher. 

In [26]:
# Commenting this out for readability but I manually looked at every pose...

# pd.set_option("display.max_rows", None)
# base_poses

In [27]:
# test
base_poses.loc[4, "Base Pose"] = "Cycling Pose"

In [28]:
base_poses

,Pose Name,Base Pose
0,Boat Pose Aerial,Boat Pose
1,Boat Pose Twist Variation Hands To Feet,Boat Pose
2,Elevated Boat Pose,Boat Pose
3,Half Bound Lotus Boat Pose,Boat Pose
4,Bicycle Pose,Cycling Pose
...,...,...
3211,Half Wind Release Pose Nose To Knee Foot Raised,Wind Release Pose
3212,Hip Opening Yoga Sequence Hippie Flow,Wind Release Pose
3213,Reclined Cow Face Pose,Wind Release Pose
3214,Seated Broken Wings Pose,Wind Release Pose


In [29]:
changes = [([4, 17, 47, 50, 60], "Cycling Pose"),
    ([19, 40, 41, 44, 46, 28, 54, 67, 68, 72], "High Boat To Low Boat Flow"),
    ([64, 66, 69, 70], "Low Boat Pose"),
    ([86], "Staff Pose"), 
    ([87, 88, 117, 118], "Bound Angle Forward Bend"), 
    ([125, 135, 136, 138, 140, 141], "One Legged Bow Pose"), 
    ([216, 217, 218, 219], "Pigeon Pose"), 
    ([233, 234, 235, 236, 237, 238, 239], "Thunderbolt Pose"), 
    ([248, 269, 270, 271, 274], "Cow Pose"), 
    ([259, 260, 261, 262, 263, 364, 265, 266, 267, 268, 272, 273, 275], "Cat Pose"),
    ([291], "Hurdlers Pose"),
    ([293, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 372], "Revolved Chair Pose"), 
    ([300, 303, 306, 307, 308], "Chair Pose With Airplane Arms"),
    ([365, 369, 370], "Figure Four Pose"), 
    ([368], "Shiva Squat Pose"), 
    ([520, 521], "Supine Spinal Twist Pose"), 
    ([587, 588], "Flying Pigeon Pose"), 
    ([594, 589], "Side Crow Pose"), 
    ([599], "Baby Crow Pose"), 
    ([663, 664, 665, 666, 667, 668], "One Legged Mountain Pose"), 
    ([687, 758], "One Handed Downward Facing Dog Pose"),
    ([688, 689, 690, 759, 760], "Standing Splits Pose"), 
    ([691, 692, 693, 694, 724, 735, 736, 737, 761, 762, 763, 764], "Three Legged Downward Facing Dog Pose"), 
    ([700, 717], "Downward Facing Dog Upward Facing Dog Pose Flow"), 
    ([714], "Downward Facing Dog Pose Plank Pose Flow"), 
    ([715], "Downward Facing Dog Pose Table Top Pose Flow"), 
    ([747], "Downward Facing Dog Pose Knee To Nose"),
    ([748], "Downward Facing Dog Pose Shoulder Taps"), 
    ([749], "Downward Facing Dog Pose to Low Lunge Pose Flow"), 
    ([865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901], "Easy Pose"), 
    ([924, 925, 926], "Cactus Arms"), 
    ([998], "Half Lotus Pose"), 
    ([999], "Lotus Pose"), 
    ([1016, 1017], "Eye Exercise"), 
    ([1055, 1056, 1057, 1058, 1061], "Scorpion Pose"), 
    ([1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151], "Chaturanga Dandasana"),
    ([1387], "Headstand Pose Eagle Legs"), 
    ([1390], "Headstand Pose Lotus Legs"), 
    ([1392], "Headstand Pose Wide Legs"), 
    ([1395, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407], "Tripod Headstand Pose"), 
    ([1464, 1465, 1479, 1480, 1481, 1482], "Revolved High Lunge Pose"), 
    ([1466, 1467, 1468, 1469, 1483], "Runners Lunge Pose"), 
    ([1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533], "Wide Legged Forward Fold"), 
    ([1487, 1509, 1510, 1511, 1512], "Revolved Wide Legged Forward Fold"),
    ([1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498], "Wide Legged Forward Fold With Halfway Lift"),
    ([1489, 1490, 1527], "Five Pointed Star Pose"), 
    ([1508], "Pyramid Pose"), 
    ([1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566], "Pyramid Pose"), 
    ([1584], "Flying Lizard Pose"), 
    ([1673, 1684, 1685, 1591, 1700, 1708, 1710, 1711, 1712], "Revolved Low Lunge Pose"),
    ([1732, 1733, 1745, 1758, 1758, 1776, 1777, 1778, 1784, 1790, 1791, 1791], "One Legged Moutain Pose"), 
    ([1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852], "Easy Pose"), 
    ([1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882], "Standing Side Bend Pose"), 
    ([1891, 1892, 1893, 1901, 1901, 1903, 1904, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922], "King Pigeon Pose"), 
    ([1927, 1928, 1932], "Grasshopper Pose"), 
    ([1929], "Dragonfly Pose"), 
    ([1930], "Eight Angle Pose"),
    ([1934, 1970, 1971, 1972], "One Legged Plank Pose"), 
    ([1939], "Side Plank Pose"), 
    ([1958, 1959, 1960, 1961, 1967, 1968, 1969], "Forearm Plank Pose"), 
    ([2173], "Lotus Pose"), 
    ([2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200], "Revolved Extended Side Angle Pose"), 
    ([2189, 2195, 2196], "Revolved High Lunge Pose"), 
    ([2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368], "Skandasana"),
    ([2390, 2397], "Anantasana"), 
    ([2415, 2416, 2417, 2418, 2419, 2420, 2443], "Visvamitrasana Pose"), 
    ([2422, 2427, 2428, 2430, 2431, 2433, 2453, 2458, 2460], "Side Plank Pose With Leg Variation"), 
    ([2436, 2437, 2438, 2439, 2440, 2441], "Forearm Side Plank Pose"), 
    ([2465, 2466], "Wild Thing Pose"), 
    ([2482, 2483, 2484, 2486, 2487, 2488, 2489, 2480, 2492, 2493], "Half Splits Pose"), 
    ([2494], "Standing Splits Pose"), 
    ([2592, 2593, 2594], "Firefly Pose"), 
    ([2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621], "Forward Fold Pose With Halfway Lift"), 
    ([2688, 2689, 2692, 2717], "Bird Of Paradise Pose"), 
    ([2700, 2701, 2703], "Revolved Hand To Big Toe Pose"), 
    ([2750, 2758], "Half Sun Salutation"), 
    ([2759], "Second Half Of Sun Salutation"), 
    ([2771, 2772], "Banana Pose"), 
    ([2795], "Supine Spinal Twist Pose"), 
    ([2797, 2798, 2833, 2848, 2849, 2850], "Tiger Pose"), 
    ([2799, 2816, 2827], "Table Top Knee To Nose Flow"), 
    ([2800, 2817], "Child Pose Table Top Pose Flow"), 
    ([2801, 2831, 2834, 2837, 2840, 2841, 2842], "Table Top Pose With One Leg Extended Back"), 
    ([2809, 2810, 2811, 2812, 2813, 2814, 2815], "Revolved Table Top Pose"), 
    ([2852, 2853, 2854, 2855], "Table Top Balancing Pose, Opposite Arm and Leg Extended"), 
    ([2968, 2972, 2983, 2974, 2975, 2976, 2977, 3005, 3006, 3007, 3008], "Revolved Triangle Pose"), 
    ([3023, 3024, 3025, 3026, 3031, 3032, 3038, 3039, 3040], "Reverse Table Top Pose"), 
    ([3037], "Flip The Dog Pose"), 
    ([3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063], "Anantasana"), 
    ([3064, 3065, 3066, 3067, 3070, 3071, 3072, 3073, 3074, 3075, 3079], "Warrior Pose I"), 
    ([3068, 3069, 3076, 3077, 3078], "Humble Warrior Pose"), 
    ([3106, 3107, 3108, 3109, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121], "Warrior Pose II"), 
    ([3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144], "Warrior Pose III"), 
    ([3134], "Shiva Squat Pose"), 
    ([3131, 3132, 3133], "Airplane Pose"), 
    ([3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190], "Half Wind Release Pose")]

In [30]:
for i in changes:
    base_poses.loc[i[0], "Base Pose"] = i[1]

In [31]:
len(base_poses["Base Pose"].unique())

170

In [32]:
base_poses["Base Pose"].unique()

array(['Boat Pose', 'Cycling Pose', 'High Boat To Low Boat Flow',
       'Low Boat Pose', 'Bound Angle Pose', 'Staff Pose',
       'Bound Angle Forward Bend', 'Bow Pose', 'One Legged Bow Pose',
       'Bridge Pose', 'Camel Pose', 'Pigeon Pose', 'Thunderbolt Pose',
       'Cow Pose', 'Cat Cow Pose', 'Cat Pose', 'Cat Tail Pulling Pose',
       'Center Splits Pose', 'Hurdlers Pose', 'Revolved Chair Pose',
       'Chair Pose', 'Chair Pose With Airplane Arms', 'Figure Four Pose',
       'Shiva Squat Pose', 'Child Pose', 'Cobra Pose', 'Corpse Pose',
       'Supine Spinal Twist Pose', 'Cow Face Pose', 'Crane Pose',
       'Flying Pigeon Pose', 'Side Crow Pose', 'Baby Crow Pose',
       'Crow Pose', 'Dancer Pose', 'One Legged Mountain Pose',
       'Dolphin Pose', 'Downward Facing Dog Pose',
       'One Handed Downward Facing Dog Pose', 'Standing Splits Pose',
       'Three Legged Downward Facing Dog Pose',
       'Downward Facing Dog Upward Facing Dog Pose Flow',
       'Downward Facing Dog P

In [33]:
base_documents = []
for i in range(len(documents)):
    temp_df = pd.DataFrame(documents[i], columns=["0"])
    temp_df = pd.merge(temp_df, base_poses, how="left", left_on="0", right_on="Pose Name")
    temp_df = temp_df.dropna(how="any")
    new_doc = [pose for pose in temp_df["Base Pose"]]
    base_documents.append(new_doc)

Running again on documents with base_poses only:

In [34]:
model = gensim.models.Word2Vec(base_documents, size=100, window=5, min_count=1, sg=1)

In [35]:
len(list(model.wv.vocab.items()))

169

In [36]:
model.most_similar("Downward Facing Dog Pose", topn=20)

[('Three Legged Downward Facing Dog Pose', 0.7480429410934448),
 ('Standing Forward Fold Pose', 0.6824171543121338),
 ('Plank Pose', 0.6556832790374756),
 ('Mountain Pose', 0.643916130065918),
 ('Forward Fold Pose With Halfway Lift', 0.6411184668540955),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.6282987594604492),
 ('Chaturanga Dandasana', 0.6114962697029114),
 ('Table Top Pose', 0.6041505932807922),
 ('Low Lunge Pose', 0.6010076403617859),
 ('Downward Facing Dog Pose Knee To Nose', 0.600443959236145),
 ('Child Pose', 0.5981735587120056),
 ('Palm Tree Pose', 0.5958209037780762),
 ('Upward Facing Dog Pose', 0.595720648765564),
 ('Reverse Warrior Pose', 0.581308126449585),
 ('Warrior Pose II', 0.5714256763458252),
 ('Table Top Balancing Pose, Opposite Arm and Leg Extended',
  0.5710283517837524),
 ('Warrior Pose I', 0.5641430616378784),
 ('Revolved High Lunge Pose', 0.5461475253105164),
 ('Downward Facing Dog Pose to Low Lunge Pose Flow', 0.5295085906982422),
 ('Sun Salutation', 0.

In [37]:
model.most_similar("Corpse Pose", topn=20)

[('Wind Release Pose', 0.7926163673400879),
 ('Supine Spinal Twist Pose', 0.7572120428085327),
 ('Happy Baby Pose', 0.7288496494293213),
 ('Half Wind Release Pose', 0.7216086387634277),
 ('Reclined Bound Angle Pose', 0.6838932037353516),
 ('Bridge Pose', 0.6568808555603027),
 ('Banana Pose', 0.6400668621063232),
 ('Eye Of The Needle Pose', 0.62393718957901),
 ('Legs Up The Wall Pose', 0.5978217720985413),
 ('Fish Pose', 0.5955902934074402),
 ('Cycling Pose', 0.5752530097961426),
 ('Section', 0.5712860822677612),
 ('Side Lying Corpse Pose', 0.5585944652557373),
 ('Reclined Big Toe Pose', 0.5263606309890747),
 ('Plough Pose', 0.514016330242157),
 ('Easy Pose', 0.4999466836452484),
 ('Seated Forward Bend Pose', 0.4835633635520935),
 ('High Boat To Low Boat Flow', 0.45954903960227966),
 ('Pranayama', 0.45049360394477844),
 ('Neck Rolls', 0.4480418860912323)]

In [38]:
model.most_similar("Wheel Pose", topn=20)

[('Shoulderstand Pose', 0.5816205739974976),
 ('Plough Pose', 0.5296584963798523),
 ('Scorpion Pose', 0.49868136644363403),
 ('Upward Plank Pose', 0.4962367117404938),
 ('Fish Pose', 0.4931376576423645),
 ('Bridge Pose', 0.46801134943962097),
 ('Bow Pose', 0.45956429839134216),
 ('Headstand Pose', 0.4423803985118866),
 ('Headstand Pose Lotus Legs', 0.43745023012161255),
 ('Reclined Hero Pose', 0.4333563446998596),
 ('Feathered Peacock Pose', 0.4309718608856201),
 ('Headstand Pose Eagle Legs', 0.4295237064361572),
 ('Eight Angle Pose', 0.42115241289138794),
 ('Reclining Hero Pose', 0.40592867136001587),
 ('Camel Pose', 0.40531104803085327),
 ('Handstand Pose', 0.4035586714744568),
 ('Flip The Dog Pose', 0.3945477306842804),
 ('Dragonfly Pose', 0.39338064193725586),
 ('Four Limbed Staff Pose', 0.39291322231292725),
 ('Headstand Pose Wide Legs', 0.390384703874588)]

In [39]:
model.most_similar("Sun Salutation")

[('Half Sun Salutation', 0.5754477977752686),
 ('Second Half Of Sun Salutation', 0.5740746259689331),
 ('Warrior Pose I', 0.5719062089920044),
 ('Triangle Pose', 0.567990779876709),
 ('Warrior Pose II', 0.5574795007705688),
 ('Mountain Pose', 0.5525914430618286),
 ('Tree Pose', 0.5489034652709961),
 ('Pranayama', 0.5420672297477722),
 ('Standing Side Bend Pose', 0.5371952652931213),
 ('Reverse Warrior Pose', 0.5239505767822266)]

In [40]:
print(model.similarity("Corpse Pose", "Extended Side Angle Pose"))
print(model.similarity("Corpse Pose", "Pigeon Pose"))
print(model.similarity("Corpse Pose", "Child Pose"))
print(model.similarity("Corpse Pose", "Wind Release Pose"))

0.11024165
0.18420866
0.41714072
0.79261637


In [41]:
model.most_similar("Wild Thing Pose")

[('Side Plank Pose', 0.6229433417320251),
 ('Side Plank Pose With Leg Variation', 0.5909644365310669),
 ('Three Legged Downward Facing Dog Pose', 0.556401252746582),
 ('Downward Facing Dog Pose Knee To Nose', 0.5535498857498169),
 ('Flip The Dog Pose', 0.5395146608352661),
 ('Visvamitrasana Pose', 0.49480071663856506),
 ('Forearm Side Plank Pose', 0.4760158061981201),
 ('Plank Pose', 0.45943447947502136),
 ('Pigeon Pose', 0.4453626871109009),
 ('Four Limbed Staff Pose', 0.43046149611473083)]

In [42]:
model.most_similar("Anantasana")

[('Side Lying Corpse Pose', 0.5074228048324585),
 ('Reverse Table Top Pose', 0.4980349540710449),
 ('Cat Tail Pulling Pose', 0.46528810262680054),
 ('Table Top Pose With One Leg Extended Back', 0.4340725243091583),
 ('Forearm Side Plank Pose', 0.42680609226226807),
 ('Cycling Pose', 0.4252551794052124),
 ('Upward Plank Pose', 0.41715162992477417),
 ('Four Limbed Staff Pose', 0.37007707357406616),
 ('Reclined Big Toe Pose', 0.3698296844959259),
 ('Side Plank Pose With Leg Variation', 0.36826905608177185)]

In [43]:
def analogy(worda, wordb, wordc):
    result = model.most_similar(negative=[worda], positive=[wordb, wordc])
    return result[0][0]

In [44]:
# A pose to a 3-limbed variation of the same pose
analogy("Downward Facing Dog Pose", "Three Legged Downward Facing Dog Pose", "Table Top Pose")

'Table Top Balancing Pose, Opposite Arm and Leg Extended'

In [45]:
# Two poses which often follow each other one after the other 
analogy("Warrior Pose II", "Extended Side Angle Pose", "Mountain Pose")

'Palm Tree Pose'

In [46]:
analogy("Warrior Pose II", "Extended Side Angle Pose", "Warrior Pose I")

'Reverse Warrior Pose'

Analysis: Now the poses have been simplfied to their base poses and hence the total possible number of input poses my application will build yoga classes from has been reduced to 170 (from 3000+). This is much more manageable, and the custom work I did to include key additional variations provides a more robust practice. 

Given the above ratings of similarity of corpse pose to other various poses, the scores make even more sense after reducing to baes poses. Very little similarity to a difficulty strength-heavy standing pose, mild similarity to an intense prone stretch pose, moderate similarity to a relaxation prone pose, and very high similarity to wind release pose, which often tends to be practiced immediately before final savasana, corpse pose. 

I am now happy with this cleaned input data and resultant word vectors demonstrating reasonable similarity, and ready to move into LSTM training. 

In [47]:
model

I'm wondering if I should have shorter vectors -- similarity to only the closest 33 poses since that's average length of a class? 

In [48]:
# model_2 = gensim.models.Word2Vec(base_documents, size=33, window=5, min_count=1, sg=1)

In [49]:
model.most_similar("Downward Facing Dog Pose", topn=20)

[('Three Legged Downward Facing Dog Pose', 0.7480429410934448),
 ('Standing Forward Fold Pose', 0.6824171543121338),
 ('Plank Pose', 0.6556832790374756),
 ('Mountain Pose', 0.643916130065918),
 ('Forward Fold Pose With Halfway Lift', 0.6411184668540955),
 ('Downward Facing Dog Pose Plank Pose Flow', 0.6282987594604492),
 ('Chaturanga Dandasana', 0.6114962697029114),
 ('Table Top Pose', 0.6041505932807922),
 ('Low Lunge Pose', 0.6010076403617859),
 ('Downward Facing Dog Pose Knee To Nose', 0.600443959236145),
 ('Child Pose', 0.5981735587120056),
 ('Palm Tree Pose', 0.5958209037780762),
 ('Upward Facing Dog Pose', 0.595720648765564),
 ('Reverse Warrior Pose', 0.581308126449585),
 ('Warrior Pose II', 0.5714256763458252),
 ('Table Top Balancing Pose, Opposite Arm and Leg Extended',
  0.5710283517837524),
 ('Warrior Pose I', 0.5641430616378784),
 ('Revolved High Lunge Pose', 0.5461475253105164),
 ('Downward Facing Dog Pose to Low Lunge Pose Flow', 0.5295085906982422),
 ('Sun Salutation', 0.

Note: if bad results/too many poses to work with (includes too many obscure poses for example) use model 3 below instead and change the min_count to filter to poses seen more often. 

In [50]:
# model_3 = gensim.models.Word2Vec(base_documents, size=100, window=5, min_count=400, sg=1)

In [51]:
# len(list(model_3.wv.vocab.items()))

In [52]:
# the 25 most common poses
model.wv.index2word[:25]

['Downward Facing Dog Pose',
 'Easy Pose',
 'Standing Forward Fold Pose',
 'Mountain Pose',
 'Corpse Pose',
 'Child Pose',
 'Cobra Pose',
 'Table Top Pose',
 'Sun Salutation',
 'Plank Pose',
 'Warrior Pose II',
 'Chair Pose',
 'Supine Spinal Twist Pose',
 'Three Legged Downward Facing Dog Pose',
 'Warrior Pose I',
 'Bridge Pose',
 'Wind Release Pose',
 'Wide Legged Forward Fold',
 'Cat Cow Pose',
 'Triangle Pose',
 'Low Lunge Pose',
 'Palm Tree Pose',
 'Staff Pose',
 'Bound Angle Pose',
 'Goddess Pose']

In [53]:
model.save("~/Metis/project_5" + "word2vec_model")

Note: Do I want/need to convert the dataset from a list of word to a list of integer indexes? 

## Preparing word embeddings for LSTM
Extracting the embedding weights from the Word2Vec embedding layer, and storing it in a numpy array for use in Keras.

In [54]:
embedding_matrix = np.zeros((len(model.wv.vocab), 100))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [55]:
embedding_matrix

array([[-0.17595953,  0.18566018, -0.11774897, ..., -0.14416806,
        -0.04487487, -0.02074035],
       [-0.22779259,  0.31133637,  0.3661651 , ..., -0.05747208,
         0.15776598,  0.01874431],
       [-0.19367634,  0.16595638, -0.1509468 , ...,  0.12080226,
        -0.08019561, -0.19894648],
       ...,
       [-0.20641275,  0.42572626, -0.07571162, ...,  0.22214788,
        -0.36750805,  0.16352049],
       [-0.11127623,  0.01856919,  0.13504539, ..., -0.13202018,
         0.19298238,  0.04338069],
       [-0.14083238,  0.2375976 ,  0.03668059, ...,  0.02468391,
         0.26796544,  0.03418195]])

In [56]:
documents[0:2]

[['Easy Pose Hands To Heart',
  'Easy Pose Hands Interlocked Behind Head',
  'Seated Torso Circles',
  'Easy Pose Variation Side',
  'Half Easy Pose Half Forward Bend',
  'Seated Windshield Wiper Pose Chest Opener Variation',
  'Cat Cow Pose',
  'Child Pose Hip Circles Flow',
  'Thread The Needle Pose Flow',
  'Downward Facing Dog Pose',
  'Downward Facing Dog Pose Plank Pose Flow',
  'Right Leg Forward',
  'Three Legged Downward Facing Dog Pose Prep',
  'Fallen Triangle Pose Variation',
  'Repeat Other Side',
  'Right Leg Forward',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Hands To Knee',
  'Low Lunge Pose Open Twist',
  'Revolved Low Lunge Pose Knee On The Floor',
  'Runners Lunge Pose',
  'Revolved Lunge Pose',
  'Extended Side Angle Pose Variation Hand In Front',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Hands To Knee',
  'Left Leg Forward',
  'Repeat Other Side',
  'Top Of Mat Section',
  'Mountain Pose',
  'Crescent High Lunge Pose Arms On Hips Flow',


In [57]:
base_documents[0:2]

[['Easy Pose',
  'Neck Rolls',
  'Easy Pose',
  'Easy Pose',
  'Easy Pose',
  'Torso Stretch Pose',
  'Cat Cow Pose',
  'Child Pose',
  'Thread The Needle Pose',
  'Downward Facing Dog Pose',
  'Downward Facing Dog Pose Plank Pose Flow',
  'Three Legged Downward Facing Dog Pose',
  'Triangle Pose',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Pose',
  'Revolved Low Lunge Pose',
  'Low Lunge Pose',
  'Revolved High Lunge Pose',
  'Extended Side Angle Pose',
  'Three Legged Downward Facing Dog Pose',
  'Low Lunge Pose',
  'Mountain Pose',
  'High Lunge Pose',
  'High Lunge Pose',
  'Warrior Pose I',
  'Warrior Pose II',
  'Reverse Warrior Pose',
  'Wide Legged Forward Fold',
  'Revolved Wide Legged Forward Fold',
  'Wide Legged Forward Fold',
  'Staff Pose',
  'Wild Thing Pose',
  'Side Plank Pose With Leg Variation',
  'Half Moon Pose',
  'Side Plank Pose',
  'Gate Pose',
  'Puppy Dog Pose',
  'Cow Face Pose',
  'Wind Release Pose',
  'Bridge Pose',
  'Supine Spinal Twist Pos

# LSTM Neural Network
Neural networks like vector input! Hooray! 


## From Medium: 
### With TensorFlow:
Turning embeddings matrix into embedding layer for TensorFlow MOdel, with "trainable=False" to aboid updating weights of embeddings layer:

In [482]:
# selecting some random poses
# valid_size = 10
# valid_window = 40
# valid_examples = np.random.choice(valid_window, valid_size, replace=False)
# valid_dataset = tf.constant(valid_examples, dtype=tf.int32) # tensorflow constant

In [484]:
# saved_embeddings = tf.constant(embedding_matrix)
# embedding = tf.Variable(initial_value=saved_embeddings, trainable=False)

Creating cosine similarity operations -- taking norm of all the embedding vectors and taking dot product between the validation words and all other word vectors: 

In [489]:
# norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
# normalized_embeddings = embedding / norm

# valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
# similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

## My Own Way: 
### Instantiating Model
Units = 33, so that output shape (size of output class) is 33 poses long, the average length of a yoga class amongst my input data. 

In [448]:
# keras_model = keras.Sequential([
#     keras.layers.Dense(units=33, use_bias=True, input_shape=(170, )),
#     keras.layers.Activation("sigmoid") # this makes is logistic regression
# ])

### Training Model 
* "sgd" = vanilla stochastic gradient descent. 
* "binary_crossentropy" = binary cross-entropy is a loss function that works when we're predicting probabilities for a binary outcome. (am I predicting a binary outcome??)

In [449]:
# keras_model.compile(optimizer="sgd", loss="binary_crossentropy")

In [513]:
# Brittany: 

# def create_model(seq_len, unique_poses, dropout=0.3, output_emb=100, rnn_unit=128, dense_unit=64):
#     inputs = tf.keras.layers.Input(shape=(seq_len,))
#     embedding = tf.keras.layers.Embedding(input_dim=unique_poses, output_dim=output_emb, input_length=seq_len)(inputs)
#     forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_unit, return_sequences=True))(embedding)
#     forward_pass , att_vector = SeqSelfAttention(
#         return_attention=True,
#         attention_activation='sigmoid',
#         attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
#         attention_width=50,
#         kernel_regularizer=tf.keras.regularizers.l2(1e-4),
#         bias_regularizer=tf.keras.regularizers.l1(1e-4),
#         attention_regularizer_weight=1e-4,
#         )(forward_pass)
#     forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
#     forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_unit, return_sequences=True))(forward_pass)
#     forward_pass , att_vector2 = SeqSelfAttention(
#         return_attention=True,
#         attention_activation='sigmoid',
#         attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL, # multiplicative
#         attention_width=50,
#         kernel_regularizer=tf.keras.regularizers.l2(1e-4),
#         bias_regularizer=tf.keras.regularizers.l1(1e-4),
#         attention_regularizer_weight=1e-4,
#         )(forward_pass)
#     forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
#     forward_pass = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(rnn_unit))(forward_pass)
#     forward_pass = tf.keras.layers.Dropout(dropout)(forward_pass)
#     forward_pass = tf.keras.layers.Dense(dense_unit)(forward_pass)
#     forward_pass = tf.keras.layers.LeakyReLU()(forward_pass)
#     outputs = tf.keras.layers.Dense(unique_poses, activation = "softmax")(forward_pass)

    # model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generate_yoga_class_rnn')
    # return model

In [515]:
# lstm = create_model(33, 169)

In [516]:
# lstm.summary()

Model: "generate_yoga_class_rnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 33)]              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 33, 100)           16900     
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 33, 256)           234496    
_________________________________________________________________
seq_self_attention_6 (SeqSel [(None, 33, 256), (None,  65537     
_________________________________________________________________
dropout_9 (Dropout)          (None, 33, 256)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 33, 256)           394240    
_________________________________________________________________
seq_self_attention_7 (SeqSel [(None, 33, 25

In [58]:
embedding_dim = model.vector_size

In [649]:
tokenizer=Tokenizer(num_words = 169)
tokenizer.fit_on_texts(base_documents)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index)
vocab_size

169

In [574]:
word_index

{'downward facing dog pose': 1,
 'easy pose': 2,
 'standing forward fold pose': 3,
 'mountain pose': 4,
 'corpse pose': 5,
 'child pose': 6,
 'cobra pose': 7,
 'table top pose': 8,
 'sun salutation': 9,
 'plank pose': 10,
 'warrior pose ii': 11,
 'chair pose': 12,
 'supine spinal twist pose': 13,
 'three legged downward facing dog pose': 14,
 'warrior pose i': 15,
 'bridge pose': 16,
 'wind release pose': 17,
 'wide legged forward fold': 18,
 'cat cow pose': 19,
 'triangle pose': 20,
 'low lunge pose': 21,
 'palm tree pose': 22,
 'staff pose': 23,
 'bound angle pose': 24,
 'goddess pose': 25,
 'extended side angle pose': 26,
 'seated forward bend pose': 27,
 'half lord of the fishes pose': 28,
 'tree pose': 29,
 'garland pose': 30,
 'reverse warrior pose': 31,
 'boat pose': 32,
 'seated straddle pose': 33,
 'pyramid pose': 34,
 'warrior pose iii': 35,
 'high lunge pose': 36,
 'locust pose': 37,
 'chaturanga dandasana': 38,
 'half moon pose': 39,
 'cycling pose': 40,
 'thunderbolt pose'

In [547]:
# convert texts to token sequences of standardized length 50
# padding fills leading zeros in or cuts off sequence at 100th pose
sequence_docs = tokenizer.texts_to_sequences(base_documents)
sequence_docs = pad_sequences(sequence_docs, maxlen=50)

In [876]:
type(sequence_docs[0][0])

numpy.int32

In [554]:
inp = Input(shape=(33,))
x = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(inp)
x = Bidirectional(LSTM(64, recurrent_dropout=.3))(x)
x = Dense(32)(x)
x = Dropout(.3)(x)
y = Dense(1, activation='softmax')(x)

NN = Model(inp, y)
NN.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 33)]              0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 33, 100)           16900     
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 128)               84480     
_________________________________________________________________
dense_17 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_14 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 33        
Total params: 105,541
Trainable params: 88,641
Non-trainable params: 16,900
____________________________________________

In [558]:
NN.compile(loss='binary_crossentropy', optimizer='adam')
history = NN.fit(sequence_docs, epochs=3, batch_size=512, verbose=10)

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: No gradients provided for any variable: ['bidirectional_19/forward_lstm_9/lstm_cell_28/kernel:0', 'bidirectional_19/forward_lstm_9/lstm_cell_28/recurrent_kernel:0', 'bidirectional_19/forward_lstm_9/lstm_cell_28/bias:0', 'bidirectional_19/backward_lstm_9/lstm_cell_29/kernel:0', 'bidirectional_19/backward_lstm_9/lstm_cell_29/recurrent_kernel:0', 'bidirectional_19/backward_lstm_9/lstm_cell_29/bias:0', 'dense_17/kernel:0', 'dense_17/bias:0', 'dense_18/kernel:0', 'dense_18/bias:0'].

# Wolfram Language
The Wolfram Language contains extensive info about 216 yoga poses, including muscle groups used, and the recommended Preparatory and Counter Poses. 

In [267]:
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession()

In [5]:
session.evaluate(wl.WolframAlpha("Easy yoga poses"))

(Rule[(('Input', 1), 'Plaintext'), 'beginner yoga poses'],
 Rule[(('Result', 1), 'Plaintext'), "Anjaneya's Pose | Balancing Table Pose | Bharadvaja's Pose | Big Toe Pose | Bikram Triangle Pose | Boat Pose | Bound Angle Pose A | Bound Angle Pose B | Bound Angle Pose C | Cat Pose | Child's Pose | Cobra Pose | Corpse Pose | Cow Pose | Crescent High Lunge Pose | Crocodile Pose | Dead Bug Pose | Deer Pose | Diamond Pose | Diamond Stretch Pose | ... (total: 90)"],
 Rule[(('Schematic:YogaData', 1), 'Plaintext'), '\n(asymmetrical poses shown with focus on the right side of the body)'],
 Rule[(('IntensityLevelSummary:YogaData', 1), 'Plaintext'), "Anjaneya's Pose | Bharadvaja's Pose | Bound Angle Pose A | Bound Angle Pose B | Bound Angle Pose C | Cat Pose | Child's Pose | Corpse Pose | Cow Pose | Crocodile Pose | Dead Bug Pose | Deer Pose | Diamond Pose | Diamond Stretch Pose | Easy Pose | Figure Four Stretch Pose | Happy Baby Pose | Head to Knee Pose A | High Lunge Pose | Knees to Chest Pose | 

In [129]:
output = session.evaluate(wlexpr('''
    EntityClass["YogaPose", 
    "PreparatoryPoses" -> 
    MemberQ[Entity["YogaPose", "EasyPose"]]] // EntityList
    '''))
output

(Entity['YogaPose', 'AccomplishedPose'], Entity['YogaPose', 'HalfLotusPose'])

In [140]:
session.evaluate(wlexpr('''
    Entity["YogaPose", "WarriorIIIPose"]["PreparatoryPoses"]
    '''))

(Entity['YogaPose', 'IntenseSideStretchPose'],
 Entity['YogaPose', 'RevolvedTrianglePose'],
 Entity['YogaPose', 'WarriorIPose'])

In [63]:
from keras.utils.data_utils import get_file
from keras.optimizers import SGD, RMSprop

In [608]:
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print(len(sentences), len(text))

# vectorization
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print("X:", X)
print("y:", y)

# build the model: a single LSTM
textgen_model = Sequential()
textgen_model.add(LSTM(128, input_shape=(maxlen, len(chars))))
textgen_model.add(Dense(len(chars)))
textgen_model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
textgen_model.compile(loss='categorical_crossentropy', optimizer=optimizer)

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
# for iteration in range(1, 60):

#     textgen_model.fit(X, y, batch_size=128, epochs=1)

#     start_index = np.random.randint(0, len(text) - maxlen - 1)

#     for diversity in [0.2, 0.5, 1.0, 1.2]:
       
#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
#         sys.stdout.write(generated)

#         for i in range(400):
#             x = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x[0, t, char_indices[char]] = 1.

#             preds = textgen_model.predict(x, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()

200285 600893
X: [[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [ True False False ... False False False]
  [ True False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 ...

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ..

In [610]:
poses = list(word_index) # word index currently sorted by how common the pose is
print('total poses:', len(poses))
pose_indices = dict((c, i) for i, c in enumerate(poses))
indices_pose = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
classes = base_documents # sentences
next_poses = [] # next_chars
for i in range(0, len(base_documents) - maxlen, step):
#     sentences.append(text[i: i + maxlen])
     next_poses.append(base_documents[i + maxlen])
# print('nb sequences:', len(sentences))

# my old vectorization method? or tokenization? 
sequence_docs = tokenizer.texts_to_sequences(base_documents)
sequence_docs = pad_sequences(sequence_docs, maxlen=50)
X = sequence_docs
y = 

# print('Vectorization...')
X = np.zeros((len(base_documents), maxlen, len(poses)), dtype=np.bool)
y = np.zeros((len(base_documents), len(poses)), dtype=np.bool)
for i, yogaclass in enumerate(classes):
    for t, pose in enumerate(yogaclass):
        X[i, t, pose_indices[pose]] = 1
    y[i, pose_indices[next_poses[i]]] = 1
print(X, y)


# # build the model: a single LSTM
print('Build model...')
textgen_model = Sequential()
textgen_model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False)(inp))
textgen_model.add(LSTM(128, input_shape=(maxlen, len(poses))))
textgen_model.add(Dense(len(chars)))
textgen_model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
textgen_model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    textgen_model.fit(X, y, batch_size=128, epochs=1)

    start_index = np.random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

    #         preds = textgen_model.predict(x, verbose=0)[0]
    #         next_index = sample(preds, diversity)
    #         next_char = indices_char[next_index]

    #         generated += next_char
    #         sentence = sentence[1:] + next_char

    #         sys.stdout.write(next_char)
    #         sys.stdout.flush()
    #     print()

total poses: 169
{'downward facing dog pose': 0, 'easy pose': 1, 'standing forward fold pose': 2, 'mountain pose': 3, 'corpse pose': 4, 'child pose': 5, 'cobra pose': 6, 'table top pose': 7, 'sun salutation': 8, 'plank pose': 9, 'warrior pose ii': 10, 'chair pose': 11, 'supine spinal twist pose': 12, 'three legged downward facing dog pose': 13, 'warrior pose i': 14, 'bridge pose': 15, 'wind release pose': 16, 'wide legged forward fold': 17, 'cat cow pose': 18, 'triangle pose': 19, 'low lunge pose': 20, 'palm tree pose': 21, 'staff pose': 22, 'bound angle pose': 23, 'goddess pose': 24, 'extended side angle pose': 25, 'seated forward bend pose': 26, 'half lord of the fishes pose': 27, 'tree pose': 28, 'garland pose': 29, 'reverse warrior pose': 30, 'boat pose': 31, 'seated straddle pose': 32, 'pyramid pose': 33, 'warrior pose iii': 34, 'high lunge pose': 35, 'locust pose': 36, 'chaturanga dandasana': 37, 'half moon pose': 38, 'cycling pose': 39, 'thunderbolt pose': 40, 'pigeon pose': 41,

KeyError: 'Easy Pose'

# Character-Based Generation : 
I'm gonna combine all the classes (all 50000 yoga classes that is) into one big document like a book like the example and just see how it does ??? *big shrug* 

In [59]:
all_classes = [pose for yogaclass in base_documents for pose in yogaclass]
print(all_classes[0:20])
text = " ".join(all_classes)
text[0:500]


['Easy Pose', 'Neck Rolls', 'Easy Pose', 'Easy Pose', 'Easy Pose', 'Torso Stretch Pose', 'Cat Cow Pose', 'Child Pose', 'Thread The Needle Pose', 'Downward Facing Dog Pose', 'Downward Facing Dog Pose Plank Pose Flow', 'Three Legged Downward Facing Dog Pose', 'Triangle Pose', 'Three Legged Downward Facing Dog Pose', 'Low Lunge Pose', 'Revolved Low Lunge Pose', 'Low Lunge Pose', 'Revolved High Lunge Pose', 'Extended Side Angle Pose', 'Three Legged Downward Facing Dog Pose']


'Easy Pose Neck Rolls Easy Pose Easy Pose Easy Pose Torso Stretch Pose Cat Cow Pose Child Pose Thread The Needle Pose Downward Facing Dog Pose Downward Facing Dog Pose Plank Pose Flow Three Legged Downward Facing Dog Pose Triangle Pose Three Legged Downward Facing Dog Pose Low Lunge Pose Revolved Low Lunge Pose Low Lunge Pose Revolved High Lunge Pose Extended Side Angle Pose Three Legged Downward Facing Dog Pose Low Lunge Pose Mountain Pose High Lunge Pose High Lunge Pose Warrior Pose I Warrior P'

In [627]:
len(text)

28839647

Since its a proof of concept, I don't want to take for*ever* running on the entire 28 million characters. giving it a subset instead. 

In [628]:
text = text[0:500000]

In [630]:
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
textgen_model = Sequential()
textgen_model.add(LSTM(128, input_shape=(maxlen, len(chars))))
textgen_model.add(Dense(len(chars)))
textgen_model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
textgen_model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    textgen_model.fit(X, y, batch_size=128, epochs=1)

    start_index = np.random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = textgen_model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

cals>.train_function at 0x7faf974bbf80> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1302/1302 [==============================] - 57s 44ms/step - loss: 0.5481

----- diversity: 0.2
----- Generating with seed: "rd Facing Dog Pose High Lunge Pose Low L"
rd Facing Dog Pose High Lunge Pose Low LWARNING:tensorflow:AutoGraph could not transform <function Model.make_predict_function.<locals>.predict_function at 0x7fafb4f23320> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source

KeyboardInterrupt: 

## Word (Pose) Based: 

Switch from char-based to word-based LM (language model)


## Create sequences:
Sequences of length 51 to train on! 

In [60]:
# tokenized, all base_poses in all 50000 yoga classes in a single list 
tokens = all_classes
print("Total tokens: ", len(tokens))
print("Unique tokens: ", len(set(tokens)))


Total tokens:  1624681
Unique tokens:  169


In [61]:
# poses from original classes (not reduced to base poses):
tokens2 = [pose for yogaclass in documents for pose in yogaclass]
print("Total tokens: ", len(tokens2))
print("Unique tokens: ", len(set(tokens2)))

Total tokens:  1906599
Unique tokens:  15372


Seeing as there are only 3600 unique yoga poses listed on the website as possible options to choose from, clearly 15000 unique 'poses' means some of what's included in the classes are unique things written by the teachers (cues, transitions, standalone variations such as "raise arms over head"... hmm). I'm interested in trying to generated classes from both. Again, the more variation the better really, expose practitioners using the app to new things, but for animation sake (as in me animated all the poses for the front end) less is better for now.... 

In [75]:
# splitting tokens into sequences of 50 input poses, 1 output pose
# so this creates tons of duplicates, and overlaps across classes since they're all one big text. this is where it would be useful to confine to single class-query 

length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    sequences.append(seq)
print("Total Sequences:", len(sequences))

Total Sequences: 1624630


In [77]:
type(sequences[0])

list

## Train language model

In [78]:
lines = sequences

In [79]:
type(lines)

list

In [81]:
tokenizer2=Tokenizer()
tokenizer2.fit_on_texts(lines)
sequences = tokenizer2.texts_to_sequences(lines)

In [82]:
vocab_size = len(tokenizer2.word_index)
vocab_size

169

In [94]:
len(sequences[0])

51

In [83]:
from keras.utils import to_categorical

In [868]:
seq_array = np.array(sequences)

In [85]:
X_array = seq_array[:,:-1]

NameError: name 'seq_array' is not defined

In [95]:
# separate into input and output.
X = [x[:-1] for x in sequences]
y = [y[-1] for y in sequences]
y = to_categorical(y, num_classes=vocab_size)
seq_length=len(X[1])

IndexError: index 169 is out of bounds for axis 1 with size 169

In [863]:
X = np.asarray(X)

In [866]:
X = tf.convert_to_tensor(X)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

## Fit Model

In [841]:
embedding_dim = 100 # weights were originally trained on 100

In [846]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
print(model.summary())

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, None, 100)         16900     
_________________________________________________________________
lstm_37 (LSTM)               (None, None, 100)         80400     
_________________________________________________________________
lstm_38 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_45 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_46 (Dense)             (None, 169)               17069     
Total params: 204,869
Trainable params: 187,969
Non-trainable params: 16,900
_________________________________________________________________
None


In [865]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=500, epochs=3, verbose=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).